In [1]:
import urllib.request
from pyspark import SparkContext 

sc = SparkContext()
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")
data_file = "kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file).sample(False, 0.1)


In [2]:
raw_data.take(1)

['0,tcp,http,SF,212,1309,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,10,0.00,0.00,0.00,0.00,1.00,0.00,0.20,21,255,1.00,0.00,0.05,0.05,0.00,0.00,0.00,0.00,normal.']

In [9]:
from pyspark.mllib.regression import LabeledPoint 
from numpy import array 

def parse_line(line):
    li = line.split(",")
    # li_values = ["42.3", "34.2", "12.3"]
    li_values = li[0: 1] + li[4: 41]
#     li_nums = [float(x) for x in li_values]
    label = li[-1]
    attack = 1.0 
    if label == "normal.":
        attack = 0.0 
    return LabeledPoint(attack, array([float(x) for x in li_values]))


In [10]:
clean_data = raw_data.map(parse_line)

494021

In [11]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

In [12]:
logit_model = LogisticRegressionWithLBFGS.train(clean_data, iterations=10)

[(0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0)]

In [21]:
ft = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz", "corrected.gz")
test_data_file = "./corrected.gz"
test_raw_data = sc.textFile(test_data_file)



In [22]:
test_data = test_raw_data.map(parse_interaction)


In [24]:
labels_and_preds = test_data.map(lambda row: (row.label, logit_model.predict(row.features)))

In [34]:
labels_and_preds.filter(lambda v: v[0] == v[1]).count()

275166

In [35]:
labels_and_preds.count()

311029

In [36]:
275166/311029.0

0.8846956393133759